In [1]:
import numpy as np
import tensorflow as tf
import models as utils
import os.path
import time
import scipy.stats as stats

In [2]:
def train_models(target_protein = 'RNCMPT00158', model_type_flag = 'CNN_struct'):
    flag = 'large'
    num_calibs = 10
    num_models = 10
#     target_protein = 'RNCMPT00158'
#     model_type_flag = 'CNN_struct'
    target_file = '../data/rnac/npz_archives/'+str(target_protein)+'.npz'
    if not(os.path.isfile(target_file)):
        utils.load_data(target_id_list=[target_protein])
    inf = np.load(target_file)
    input_configuration = utils.input_config(flag)
    config_list = utils.generate_configs(num_calibs, model_type_flag, flag)
    input_data = utils.Deepbind_input(input_configuration, inf, model_type_flag)
    models = []
    Nbatch_train = input_data.training_cases // 100
    pearson_test =np.zeros(num_models)
    print(target_protein, model_type_flag)
    with tf.Graph().as_default():
        for i in range(num_calibs):
            models.append(utils.Deepbind_model(config_list[i], input_data, model_type_flag))
        start_time = time.time()
        with tf.Session() as session:
            session.run(tf.initialize_all_variables())
            for epoch in range(10):
                _ = utils.run_epoch_parallel(session, models, input_data, config_list[0], epoch, train=True)
    #             for step in range(Nbatch_train):
    #                 mbatchX_train = input_data.training_data[(100* step) : (100 * (step+1)),:,:]
    #                 mbatchY_train = input_data.training_labels[(100*step):(100*(step+1))]
    #                 feed_dict = {}
    #                 fetches = {}
    #                 for i in range(num_calibs):
    #                     feed_dict[models[i].x] = mbatchX_train
    #                     feed_dict[models[i].y_true] = mbatchY_train
    #                     fetches["cost"+str(i+1)] = models[i].cost
    #                     fetches["eval_op" +str(i+1)] = models[i].train_op
    #     #             with tf.Session() as sess:
    #     #                 sess.run(tf.initialize_all_variables())
    #                 vals = sess.run(fetches, feed_dict)
            _,_,pearson_test = utils.run_epoch_parallel(session, models, input_data, config_list[0], epoch, train=False, verbose=True, testing=True)
    #         feed_dict = {}
    #         fetches = {}
    #         for i in range(num_calibs):
    #             feed_dict[models[i].x] = input_data.test_data
    #             feed_dict[models[i].y_true] = input_data.test_labels
    #             fetches["predictions"+str(i)] = models[i].predict_op
    # #             fetches["eval_op" +str(i+1)] = models[i].train_op
    #         vals = sess.run(fetches, feed_dict)
    #         for j in range(num_models):
    # #             cost_test[j] = vals["cost"+str(j)]
    #             pearson_test[j] = stats.pearsonr(input_data.test_labels, vals["predictions"+str(j)])[0]
    #         print(pearson_test)
        elapsed_time = time.time() - start_time
    # print(elapsed_time)
    return np.max(pearson_test)


In [3]:
# protein_list = ['RNCMPT00289',
# 'RNCMPT00158',
# 'RNCMPT00258',
# 'RNCMPT00081',
# 'RNCMPT00147',
# 'RNCMPT00199',
# protein_list=[
# 'RNCMPT00209',
# 'RNCMPT00095',
# 'RNCMPT00146',
# 'RNCMPT00149',
# 'RNCMPT00161',
# ]
protein_list = [
# 'RNCMPT00278',
# 'RNCMPT00172',
'RNCMPT00035',
'RNCMPT00270',
'RNCMPT00085',
'RNCMPT00202',
'RNCMPT00251',
'RNCMPT00154',
]
pearson_struct = []
pearson_nostruct = []
for protein in protein_list:
    pearson_test_CNN = train_models(protein, 'CNN')
    save_target = str(protein) +'CNN.npz'
    np.savez(save_target, pearson = pearson_test_CNN)
    save_target = str(protein) + 'CNN_struct.npz'
    pearson_test_CNN_struct = train_models(protein, 'CNN_struct')
    pearson_nostruct.append(pearson_test_CNN)
    pearson_struct.append(pearson_test_CNN_struct)

('RNCMPT00035', 'CNN')
Epoch:0010, Train cost(avg)=1.0549, Test cost(avg)=1.0581, Test Pearson(avg)=0.1606
[ 0.16077195  0.16283578  0.15966233  0.15283232  0.16439441  0.16371847
  0.16349339  0.15204737  0.16341538  0.16265375]
('RNCMPT00035', 'CNN_struct')
Epoch:0010, Train cost(avg)=1.0434, Test cost(avg)=1.0481, Test Pearson(avg)=0.1830
[ 0.20019813  0.20668191  0.18312544  0.182998    0.11191582  0.21250588
  0.16867448  0.18327492  0.19120521  0.18972923]
('RNCMPT00270', 'CNN')
Epoch:0010, Train cost(avg)=0.8731, Test cost(avg)=0.8823, Test Pearson(avg)=0.1912
[ 0.19237443  0.18603633  0.19128832  0.2033844   0.19199624  0.20248646
  0.20304405  0.1850025   0.15876266  0.19767957]
('RNCMPT00270', 'CNN_struct')
Epoch:0010, Train cost(avg)=0.8739, Test cost(avg)=0.8828, Test Pearson(avg)=0.1893
[ 0.17800095  0.20302639  0.19154477  0.19137566  0.19682547  0.17439875
  0.19641017  0.16343555  0.19332943  0.20505879]
('RNCMPT00085', 'CNN')
Epoch:0010, Train cost(avg)=2.0425, Test co

In [4]:
protein_list = [
# 'RNCMPT00278',
# 'RNCMPT00172',
'RNCMPT00035',
'RNCMPT00270',
'RNCMPT00085',
'RNCMPT00202',
'RNCMPT00251',
'RNCMPT00154',
]
print("Protein\t\tCNN\t\tCNN_s\t\tDelta")
for i in range(len(protein_list)):
    improvement = (pearson_struct[i] - pearson_nostruct[i])*100 / (pearson_nostruct[i])
    print("%s\t\t %.6f \t\t %.6f\t\t %.2f%%"%(protein_list[i], pearson_nostruct[i], pearson_struct[i], improvement))

Protein		CNN		CNN_s		Delta
RNCMPT00035		 0.164394 		 0.212506		 29.27%
RNCMPT00270		 0.203384 		 0.205059		 0.82%
RNCMPT00085		 0.353817 		 0.391389		 10.62%
RNCMPT00202		 0.461906 		 0.517447		 12.02%
RNCMPT00251		 0.226518 		 0.237175		 4.70%
RNCMPT00154		 0.183320 		 0.195352		 6.56%


In [5]:
protein_list = ['RNCMPT00289',
'RNCMPT00158',
'RNCMPT00258',
'RNCMPT00081',
'RNCMPT00147',
'RNCMPT00199',
'RNCMPT00209',
'RNCMPT00095',
'RNCMPT00146',
'RNCMPT00149',
'RNCMPT00161',
'RNCMPT00278',
'RNCMPT00172',
'RNCMPT00283',
'RNCMPT00262',
'RNCMPT00255',
'RNCMPT00020',
'RNCMPT00005',
'RNCMPT00076',
]
pearson_struct = []
pearson_nostruct = []
for i, protein in protein_list:
    start_time = time.time()
    pearson_nostruct.append(train_models(protein, 'CNN'))
    pearson_struct.append(train_models(protein, 'CNN_struct'))

ValueError: too many values to unpack